Laad nuttige functionaliteit

In [ ]:
# Creeren van figuren 
using Plots, LaTeXStrings


# Kleine verzameling van PDE voorbeelden gebaseerd op BasisFunctions.jl en FrameFun.jl
using FrameFunApplications 

# Stroming rond een schijf

We berekenen numeriek de stroming rond een schijf

In [ ]:
opties = (;R =.1,N=200)

In [ ]:
sol = run_circle_example(;opties...);

In [ ]:
gr();quiver_plot(15,sol.u,sol.domain;γ=9,xlims=(.2,.8),ylims=(.2,.8),size=(300,300),
    linewidth=2,cbar=false,ratio=1,xlabel="x",ylabel="y",background_color = :transparent,foreground_color=:black)
# savefig("potentialflow")

## Stap voor stap

In [ ]:
# R is de straal van de schijf, N^2 het aantal functies waarmee wordt benaderd
opties = (;R =.1,N=200)
# Een set van testfuncties waarmee we de oplossing willen benaderen (hier later meer over)
basis = spline_basis(;opties...)
# Maak het gebruik van de symbolen voor partieel afgeleiden mogelijk 
using SymbolicDifferentialOperators: δx, δy, Δ, I
# Uniforme layout van figuren
plot_opties = (;xlabel="x",
                ylabel="y",
                xlims=(0,1),
                ylims=(0,1),
                markersize = 1,
                markerstrokewidth = 0.,
                ratio=1,
                size=(300,300)
            );

### Stap 1. Leg PDE op in het inwendige 

In het inwendige ($\Omega$) (zie het rooster hieronder) leggen we de partiele differentiaal vergelijking op 

$$ \Delta u(x,y) = 0\quad $$
waarbij $\Delta = \frac{\partial^2}{\partial^2x} + \frac{\partial^2}{\partial^2y}$.

In [ ]:
stap_1 = PDERule(
        Δ,
        basis,
        collocatie_rooster(;opties...),
        (x,y)->0.
    );

In [ ]:
plot(collocatie_rooster(;opties...,N=30);plot_opties...)

### Stap 2. Water mag niet door verticale wanden of schijf vloeien

<b>Randvoorwaarden in plaats van beginvoorwaarden!</b>

In [ ]:
plot(wand_randvoorwaarde_rooster(;opties...);plot_opties...)
plot!(schijf_rooster(;opties...);plot_opties...)

In [ ]:
stap_2a = PDERule(
            # Snelheid (afgeleide) in de horizontale richting ... 
            δx^1*δy^0,
            basis,
            wand_randvoorwaarde_rooster(;opties...),
            # ... is nul.
            (x,y)->0.
        );
stap_2b = PDENormalRule(
            # Snelheid (afgeleide) loodrecht op de schijf ... 
            schijf(;opties...), 
            basis,
            schijf_rooster(;opties...),
            # ... is nul.
            (x,y)->0
        );

### Stap 3. Water vloeit verticaal aan horizontale uiteindes

In [ ]:
plot(stroming_randvoorwaarde_rooster(;opties...);plot_opties...)

In [ ]:
stap_3 = PDERule(
            # Snelheid (afgeleide) in de verticale richting ... 
            δy^1*δx^0,
            basis,
            stroming_randvoorwaarde_rooster(;opties...),
            # heeft snelheid v
            (x,y)->arg(:v,opties)
        );

### Alles samenvoegen en oplossen

In [ ]:
using FrameFunTranslates: sparseQR_solver

pde = PDE(stap_1,stap_2a,stap_2b,stap_3)
u = pdesolve(pde;directsolver=sparseQR_solver)

In [ ]:
gr();quiver_plot(15,u,domein(;opties...);γ=9,linewidth=2,cbar=false,ratio=1,size=(300,300))

# Analytische oplossing (met behulp van complexe functieleer)

Een schijf heeft een envoudige vorm daarom bestaat er ook een analytische oplossing voor de PDE. 

In [ ]:
R = arg(:r,opties)
function analytische_oplossing(z::Complex)
    z + R^2/z
end

In [ ]:
# Onderstaande code zorgt dat `analytische_oplossing` zich gedraagt als een PDE-oplossing.
import BasisFunctions: diff, dimension
using GridArrays
function analytische_oplossing_dy(z::Complex) 
    1 - R^2/(z^2)
end
function analytische_oplossing_dx(z::Complex) 
    1im - 1im*R^2/(z^2)
end
for fun in (:analytische_oplossing,:analytische_oplossing_dx,:analytische_oplossing_dy)
    @eval begin 
        $fun((y,x),) = real($fun(complex(x,y)-complex(.5,.5)))
        $fun(g::AbstractGrid) = $fun.(g)
        dimension(::typeof($fun)) = 2
    end
end
function diff(::typeof(analytische_oplossing),order)
    if order==(1,0)
        analytische_oplossing_dx
    elseif order==(0,1)
        analytische_oplossing_dy 
    end
end;

In [ ]:
gr();quiver_plot(15,analytische_oplossing,domein(;opties...);γ=9,linewidth=2,cbar=false,ratio=1,size=(300,300))

# Stroming rond vierkant

In [ ]:
sol_full = FrameFunApplications.SplineSquareExample._run_square_example(;
    N=400,γ=2,r=sqrt(2)/20);
sol_quarter = FrameFunApplications.SplineSquareExample._run_quarter_square_example_sing(;
    N=400,γ=2,r=.17,sing=false);
sol_quarter_sing = FrameFunApplications.SplineSquareExample._run_quarter_square_example_sing(;
    N=400,γ=2,r=.17,sing=true);

In [ ]:
quiver_plot(17,sol_full.u,sol_full.domain;γ=9,linewidth=2,cbar=false,ratio=1,
    xlims=(.15,.85),ylims=(.15,.85),size=(300,300),background_color = :transparent,foreground_color=:black)
# savefig("squareflow")

In [ ]:
quiver_plot(17,sol_quarter_sing.u,sol_quarter_sing.domain;γ=9,linewidth=2,cbar=false,
    ratio=1,xlims=(.15,.85),ylims=(.15,.85),size=(300,300),background_color = :transparent,foreground_color=:black)
# savefig("quarterflow")

In [ ]:
sol_quarter.coefficient_norm_inf, sol_quarter_sing.coefficient_norm_inf

In [ ]:
sol_quarter.coefficient_norm_l2, sol_quarter_sing.coefficient_norm_l2

In [ ]:
sol_quarter.rel_residual_l2, sol_quarter_sing.rel_residual_l2

In [ ]:
sol_quarter.rel_residual_inf, sol_quarter_sing.rel_residual_inf

In [ ]:
sol_quarter.abs_residual_l2, sol_quarter_sing.abs_residual_l2

In [ ]:
sol_quarter.abs_residual_inf, sol_quarter_sing.abs_residual_inf

In [ ]:
using BasisFunctions
(x,y),M = heatmap_matrix(element(dictionary(sol_quarter_sing.u),2)[1],(0.0..1.0)^2,EquispacedGrid(101,0,1)^2)
heatmap(x,y,M,c=:blues,background_color = :transparent,foreground_color=:black)
contour!(x,y,M,c=:black;xlims=(.2,.8),ylims=(.2,.8),ratio=1,cbar=false,size=(300,300))
# savefig("singflow")